# Precision: Calculating how many of the extracted relationships are valid

In [7]:
import nltk
from nltk.stem import PorterStemmer
nltk.download("punkt")

# Initialize Python porter stemmer
ps = PorterStemmer()
def lemmatize(sent):
    return [ps.stem(word) for word in sent.split()]

AttributeError: partially initialized module 'nltk' has no attribute 'data' (most likely due to a circular import)

In [6]:
!pip install -U nltk

  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)
  Attempting uninstall: nltk
    Found existing installation: nltk 3.8.1
    Uninstalling nltk-3.8.1:
      Successfully uninstalled nltk-3.8.1

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [8]:
import fitz,os
from sentence_transformers import SentenceTransformer, util

# Load a pre-trained model
model = SentenceTransformer('all-MiniLM-L6-v2')
import nltk
from nltk.stem import PorterStemmer
nltk.download("punkt")

# Initialize Python porter stemmer
ps = PorterStemmer()
def lemmatize(sent):
    return [ps.stem(word) for word in sent.split()]
#reading the pdf and hand annotation files
def read_pdf(pdf_file):
    start=False
    sentences=[]
    start_idx=0
    with fitz.open(pdf_file) as pdf_file:
        for page_index, page in enumerate(pdf_file):
            text = page.get_text("text").lower()
            text=text.split(". ")
            sentences.extend(text)
                
    return sentences
def read_files(root_dir, hand):
    
    lines=[]
    for files in os.listdir(root_dir):
        if files[-4:] != '.pdf':
            continue
        sentences = read_pdf(f"{root_dir}/{files}")
        lines.extend(sentences)

    # read in hand annotations
    for p in hand.iterrows():
        rel = p[1]['rel']
        subj = p[1]['subj']
        obj = p[1]['obj']
        out=f"{subj} {rel} {obj}" 
        lines.append(out)


    return lines

#computing cosine similarity
def vec(sentences):
    # Encode sentences
    embeddings = model.encode([sentences[0], sentences[1]])
    
    # Compute cosine similarity
    similarity = util.cos_sim(embeddings[0], embeddings[1])
    return similarity.item() # Value close to 1 indicates high similarity
    
#finding if the target string (relation triplet) is in the src (pdf + hand annotation)
def find(target, src):
    found=False
    matching_sentence=""
 
    for idx,sentence in enumerate(src):
        pred=" ".join(lemmatize(target))
        test=" ".join(lemmatize(sentence))
        cos = vec([pred,test])
        if pred in test or cos > 0.7:
            if cos >0.65 and cos < 0.7:
                print(f"Got a match for {pred }: {sentence}")
            elif cos <=0.65:
                print(f"Closest match to {pred} was {test}")
            found=True
            st_idx=idx
            matching_sentence=sentence
            return found, matching_sentence, 
            
    return found, matching_sentence


KeyboardInterrupt: 

In [ ]:
print(f"Example of a sentence from the pdf: {sentences[4]}\n\n")
print(f"Example from the hand annot. {sentences[-1]}")

In [6]:
#find target in hand annotations
def find_in_hand(hand,target):
    max_=-1.0
    for row in hand.iterrows():
        hand_rel = f" {row[1]['subj']} {row[1]['rel']} {row[1]['obj']}"
        sub_h = row[1]['subj']
        cos=vec([hand_rel, target])
        
        if cos > max_:
            max_ = cos
            best=hand_rel
        
        cos=vec([best, hand_rel])
        if sub_h.lower() not in best.lower() or cos < 0.7:
            print(f"Closest match for {hand_rel} is {best} with {cos}")
            continue
        else:
            return best
            #print(f"Closest match for {hand_rel} is {best} with {vec([best, hand_rel])}")
            #print(rouge_([hand_rel], [best]))
    

In [1]:

import pandas as pd
hand = pd.read_csv("../Results/ground_truth.csv")
pred_files = os.listdir("../Results")
                        
sentences = read_files("../Docs",hand)


for pred_file in ['Temperature1_WithoutExamples.csv']:
    if any(substring in pred_file for substring in [ 'cumm', '.ipynb', 'WithoutEx', 'ground', 'Clean', 'result']):
            continue
    pred_file = f"../Results/{pred_file}"
    preds = pd.read_csv(pred_file) 
    score = 0
    for p in preds.iterrows():
        ref = p[1]['ref']
        rel = p[1]['rel']
        subj = p[1]['subj']
        obj = p[1]['obj']
        out=f"{subj} {rel} {obj}"
        
        found, match = find(out, sentences)
        if found:
            score +=1
        else:
            print("Couldn't find a match for  ", out)
    print(f"Precision for {pred_file} is {score/len(preds)}")

NameError: name 'os' is not defined

In [ ]:
print(f"Recall score: {100*score/len(preds)}%")

In [ ]:
Precision for ../NewRels_Skip3_PassingInIncrements.csv : 0.8181818181818182
Precision for ../NewRels_Skip4_increments.csv is 0.6785714285714286
Precision for ../Results/NewRels_Skip3_cummulative.csv is 0.8584905660377359
Precision for ../Results/NewRels_Skip2_cummulative.csv is 0.901840490797546
Precision for ../Results/NewRels_Skip2_increments.csv is 0.7616279069767442
Precision for ../Results/Temperature0point2.csv is 0.7981651376146789